In [132]:
using DataFrames, CSV, Dates, JLD2

trips = CSV.read("../../data/trips/202209-bluebikes-tripdata.csv", DataFrame);
station_information = CSV.read("../../data/stations/station_information.csv", DataFrame);

# Trips 

In [133]:
# find unique station_id
station_ids = unique(station_information[:, :station_id]);
nb_stations = length(station_ids);
stations_match = Dict(zip(station_ids, 1:nb_stations));

# eliminate trips with station_id not in station_ids
trips = trips[in.(trips[:, :"start station id"], [Set(station_ids)]), :];
trips = trips[in.(trips[:, :"end station id"], [Set(station_ids)]), :];

# convert starttime and stoptime to DateTime
trips[!, :starttime] = DateTime.(trips[!, :starttime], "yyyy-mm-dd HH:MM:SS.ssss");
trips[!, :stoptime] = DateTime.(trips[!, :stoptime], "yyyy-mm-dd HH:MM:SS.ssss");

# extract day and hour from starttime and stoptime
trips[!, :startday] = Dates.day.(trips[!, :starttime]);
trips[!, :starthour] = Dates.hour.(trips[!, :starttime]);
trips[!, :stopday] = Dates.day.(trips[!, :stoptime]);
trips[!, :stophour] = Dates.hour.(trips[!, :stoptime]);

# create matrix of hourly trips between stations_match
nb_days = maximum(trips[:, :startday]);
trips_matrix = zeros(Int, nb_stations, nb_stations, nb_days*24);
for i in 1:size(trips, 1)
    trips_matrix[stations_match[trips[i, :"start station id"]], stations_match[trips[i, :"end station id"]], (trips[i, :startday]-1)*24+trips[i, :starthour]+1] += 1
end

In [134]:
# jldsave("../../data/parameters/202209-trips-matrix.jld2", true; trips_matrix)

In [135]:
trips_matrix = load("../../data/parameters/202210-trips-matrix.jld2", "trips_matrix");

# Capacity

In [136]:
capacity = station_information[:, :capacity];

# Demand

In [137]:
function estimate_demand(trips_matrix)
    # if number of trips is more than 75% of capacity, corresponding number is quadrupled
    nb_stations, _, nb_hours = size(trips_matrix)
    demand = copy(trips_matrix)
    for k in 1:nb_hours

        # outgoing demand
        for i in 1:nb_stations
            if sum(trips_matrix[i, :, k]) > capacity[i]
                total_outgoing = sum(trips_matrix[i, :, k])
                for j in 1:nb_stations
                    demand[i, j, k] += ceil(trips_matrix[i, j, k] - trips_matrix[i, j, k] * capacity[i]/total_outgoing)
                end
            end
        end

        # incoming demand
        for j in 1:nb_stations
            if sum(trips_matrix[:, j, k]) > capacity[j]
                total_incoming = sum(trips_matrix[:, j, k])
                for i in 1:nb_stations
                    demand[i, j, k] += ceil(trips_matrix[i, j, k] - trips_matrix[i, j, k] * capacity[j]/total_incoming)
                end
            end
        end
    end
    return demand
end

estimate_demand (generic function with 1 method)

In [138]:
demand = estimate_demand(trips_matrix);

In [139]:
jldsave("../../data/parameters/202210-demand.jld2", true; demand)